### Carzam (CompCars)
Trying to do transfer learning on the Vgg16 model for predicting car make/model using the CompCars dataset.

In [ ]:
import os
import glob

import keras
from keras.preprocessing import image

from matplotlib import pyplot as plt
import numpy as np
from numpy.random import permutation
np.set_printoptions(precision=4, linewidth=100)
from sklearn.cross_validation import train_test_split
import skimage
from skimage import data, color, exposure

In [ ]:
# Contains some utilty functions
# TODO Figure out what "reload" does
import utils; reload(utils)
from utils import plots
# Contains the trained Vgg16 model (2014 winner of ImageNet)
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [ ]:
path = "../data/compcars/data/image/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [ ]:
# For showing the plots in this webpage
%matplotlib inline

In [ ]:
# If running out of memory or using an older GPU, decrease this number
batch_size = 32

In [ ]:
vgg = Vgg16()

In [ ]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

In [ ]:
imgs, labels = next(batches)

In [ ]:
plots(imgs, titles=labels)

In [ ]:
vgg.finetune(batches)

In [ ]:
# Plot the results of the training
def plot_results(history):
    # Summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
vgg.model.load_weights(model_path+'.h5')

In [ ]:
vgg.predict(imgs, True)

In [ ]:
test_batches, probs = vgg.test(path+'test', batch_size=batch_size)
labels = test_batches.classes
filenames = test_batches.filenames

In [ ]:
labels_predicted = [np.argmax(prob) for prob in probs]

In [ ]:
test_imgs = next(test_batches)
plots(test_imgs, titles=labels_predicted)

In [ ]:
labels_predicted

In [ ]:
classes = [vgg.classes[idx] for idx in labels_predicted]

In [ ]:
filenames

In [ ]:
def check_count_correct(filenames, classes):
    assert(len(filenames) == len(classes))
    correct = incorrect = 0
    for i in range(len(filenames)):
        if filenames[i].split('/')[0] == classes[i]:
            correct += 1
        else:
            incorrect += 1
    return correct, incorrect

In [ ]:
correct, incorrect = check_count_correct(filenames, classes)
print correct
print incorrect

In [ ]:
batches.classes

In [ ]:
def plots_idx(idx, titles=None):
    """Helper function for plotting images by index in the validation set."""
    if len(idx) > 0:
        plots([image.load_img(path+'valid/'+filenames[i]) for i in idx], titles=titles)
    else:
        print "Nothing to plot!"